In [5]:
!pip install -q -U google-generativeai

In [3]:
import pathlib
import textwrap

import google.generativeai as genai
import numpy as np
import pandas as pd

from IPython.display import display
from IPython.display import Markdown

In [7]:
from config import GOOGLE_API_KEY

genai.configure(api_key=GOOGLE_API_KEY)

In [8]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


In [9]:
model = genai.GenerativeModel('gemini-pro')

In [14]:
%%time
response = model.generate_content("What is the meaning of life?")

CPU times: user 57 ms, sys: 5.14 ms, total: 62.1 ms
Wall time: 4.8 s


In [7]:
response = model.generate_content("code in version 4 comprised of 5 codes in version 3.1, 2691 which is (Manufacture of non-structural non-refractory ceramic ware), code 2696 which is (Cutting, shaping and finishing of stone), 2692 which is (Manufacture of refractory ceramic products), code 2693 which is (Manufacture of structural non-refractory clay and ceramic products), code 2694 which is (Manufacture of cement, lime and plaster), code 2699 (Manufacture of other non-metallic mineral products n.e.c.), and code 2695 which is (Manufacture of articles of concrete, cement and plaster). What is your best estimate of the proportion of jobs now coded in 239 that were in each of the previous codes in version 3.1? Can you give me you best guesses in a table?")
response.text

'| Previous Code in Version 3.1 | Proportion of Jobs Now Coded in 239 |\n|---|---|\n| 2691 (Manufacture of non-structural non-refractory ceramic ware) | 20% |\n| 2696 (Cutting, shaping and finishing of stone) | 15% |\n| 2692 (Manufacture of refractory ceramic products) | 10% |\n| 2693 (Manufacture of structural non-refractory clay and ceramic products) | 25% |\n| 2694 (Manufacture of cement, lime and plaster) | 15% |\n| 2699 (Manufacture of other non-metallic mineral products n.e.c.) | 10% |\n| 2695 (Manufacture of articles of concrete, cement and plaster) | 5% |'

In [8]:
from io import StringIO
import csv

# Define the data string
data_string = response.text

# Create a StringIO object to mimic a file-like object
data_io = StringIO(data_string)
print(data_io)

In [9]:
df_2 = pd.read_csv(data_io, delimiter="|")
df_2 = df_2.drop(index = [0])
df_2


,Unnamed: 0,Previous Code in Version 3.1,Proportion of Jobs Now Coded in 239,Unnamed: 3
1,NaN,2691 (Manufacture of non-structural non-refra...,20%,NaN
2,NaN,"2696 (Cutting, shaping and finishing of stone)",15%,NaN
3,NaN,2692 (Manufacture of refractory ceramic produ...,10%,NaN
4,NaN,2693 (Manufacture of structural non-refractor...,25%,NaN
5,NaN,"2694 (Manufacture of cement, lime and plaster)",15%,NaN
6,NaN,2699 (Manufacture of other non-metallic miner...,10%,NaN
7,NaN,"2695 (Manufacture of articles of concrete, ce...",5%,NaN


In [10]:
names = df_2.columns
df_2 = df_2.drop(columns = [names[0], names[-1]])
df_2

,Previous Code in Version 3.1,Proportion of Jobs Now Coded in 239
1,2691 (Manufacture of non-structural non-refra...,20%
2,"2696 (Cutting, shaping and finishing of stone)",15%
3,2692 (Manufacture of refractory ceramic produ...,10%
4,2693 (Manufacture of structural non-refractor...,25%
5,"2694 (Manufacture of cement, lime and plaster)",15%
6,2699 (Manufacture of other non-metallic miner...,10%
7,"2695 (Manufacture of articles of concrete, ce...",5%


In [11]:
print(names)
print(df_2.columns)

Index(['Unnamed: 0', ' Previous Code in Version 3.1 ',
       ' Proportion of Jobs Now Coded in 239 ', 'Unnamed: 3'],
      dtype='object')
Index([' Previous Code in Version 3.1 ', ' Proportion of Jobs Now Coded in 239 '], dtype='object')


In [12]:
#df_2.reset_index()
df_2[names[2]] = df_2[names[2]].str.strip()
df_2[names[2]] = df_2[names[2]].str.replace("%", "")
df_2[names[2]] = df_2[names[2]].astype(float) / 100
#type(df_2[names[3]][3])
df_2

,Previous Code in Version 3.1,Proportion of Jobs Now Coded in 239
1,2691 (Manufacture of non-structural non-refra...,0.20
2,"2696 (Cutting, shaping and finishing of stone)",0.15
3,2692 (Manufacture of refractory ceramic produ...,0.10
4,2693 (Manufacture of structural non-refractor...,0.25
5,"2694 (Manufacture of cement, lime and plaster)",0.15
6,2699 (Manufacture of other non-metallic miner...,0.10
7,"2695 (Manufacture of articles of concrete, ce...",0.05


In [13]:
pattern = r'\D'  # Matches any non-digit character

# Replace non-numeric characters with an empty string
df_2[names[1]] = df_2[names[1]].str.replace(pattern, '')
df_2

<ipython-input-13-13a2a72d999b>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_2[names[1]] = df_2[names[1]].str.replace(pattern, '')


,Previous Code in Version 3.1,Proportion of Jobs Now Coded in 239
1,2691,0.20
2,2696,0.15
3,2692,0.10
4,2693,0.25
5,2694,0.15
6,2699,0.10
7,2695,0.05


In [ ]:
def get_proportions(four_code, three_code):
  from io import StringIO
  import csv
